Variable Scope and Binding

Nonlocal Variables

In [2]:
# Example is to create function that can increment
def counter():
    num = 0
    def incrementer():
        num += 1
        return num
    return incrementer
c = counter()
c()

UnboundLocalError: local variable 'num' referenced before assignment

In [3]:
# When nonlocal is added to mix
def counter():
    num = 0
    def incrementer():
        nonlocal num
        num += 1
        return num
    return incrementer
c = counter()
c()

1

In [4]:
c()

2

In [5]:
c()

3

nonlocal will allow you to assign to variables in an outer scope, but not a global scope. So you can't use nonlocal in our counter function because then it would try to assign to a global scope. Give it a try and you will quickly get a SyntaxError. Instead you must use nonlocal in a nested function.

Global Variables

In [6]:
x = 'Hi'
def read_x():
    print(x)  # x is just referenced, therefore assumed global
read_x()  # prints Hi
def read_y():
    print(y)  # here y is just referenced, therefore assumed global
read_y()  # NameError: global name 'y' is not defined

Hi


NameError: name 'y' is not defined

In [7]:
def read_y():
    y = 'Hey' # y appears in an assignment, therefore it's local
    print(y)
read_y()    # prints Hey
def read_x_local_fail():
    if False:
        x = 'Hey' # x appears in an assignment, therefore it's local
    print(x)  # will look for the _local_ z, which is not assigned, and will not be found
read_x_local_fail()  # UnboundLocalError

Hey


UnboundLocalError: local variable 'x' referenced before assignment

In [9]:
# an assignment inside a scope will shadow any outer variables of the same name
x = 'Hi'
def change_local_x():
    x = 'Bye'
    print(x)
change_local_x()  # prints Bye
print(x)

Bye
Hi


In [10]:
x = 'Hi'
def change_global_x():
    global x
    x = 'Bye'
    print(x)
change_global_x() # prints Bye
print(x) # prints Bye

Bye
Bye


Local Variables

In [11]:
def foo():
    a = 5
    print(a) # ok
print(a) # NameError

NameError: name 'a' is not defined

In [13]:
def foo():
    if True:
        a = 5
    print(a)
foo()
b = 3
def bar():
    if False:
        b = 5
    print(b)
bar()

5


UnboundLocalError: local variable 'b' referenced before assignment

The del command

In [14]:
# del removes the variable from its scope
x = 5
print(x)

5


In [15]:
del x
print(x)

NameError: name 'x' is not defined

In [16]:
class A:
    pass
a = A()
a.x = 7
print(a.x)

7


In [17]:
del a.x
print(a.x)

AttributeError: 'A' object has no attribute 'x'

In [20]:
x = {'a': 1, 'b':2}
del x['a']
print(x)

{'b': 2}


In [21]:
print(x['a'])

KeyError: 'a'

In [22]:
x = [0, 1, 2, 3, 4]
del x[1:3]
print(x)

[0, 3, 4]


Functions skip class scope when looking up
names

In [24]:
a = 'global'
class Fred:
    a = 'class' # class scope
    b = (a for i in range(10)) # function scope
    c = [a for i in range(10)]
    d = a # class scope
    e = lambda: a # function scope
    f = lambda a=a: a # default argument uses class scope
    @staticmethod # or @classmethod, or regular instance method
    def g(): # function scope
        return a
print(Fred.a) # class
print(next(Fred.b)) # global
print(Fred.c[0]) # global in python 3
print(Fred.d) # class
print(Fred.e()) # global
print(Fred.f()) # class
print(Fred.g()) # global

class
global
global
class
global
class
global


Local vs Global Scope

In [26]:
foo = 1 # global
def func():
    bar = 2  #local
    print(foo) # prints variable foo from global scope
    print(bar) # prints variable bar from local scope
func()

1
2


In [27]:
foo = 1
def func():
    bar = 2
    print(globals().keys()) # prints all variable names in global scope
    print(locals().keys()) # prints all variable names in local scope
func()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'counter', '_1', '_i2', 'c', '_i3', '_3', '_i4', '_4', '_i5', '_5', '_i6', 'read_x', 'read_y', '_i7', 'read_x_local_fail', '_i8', 'change_local_x', '_i9', '_i10', 'change_global_x', '_i11', 'foo', '_i12', 'b', 'bar', '_i13', '_i14', '_i15', '_i16', 'A', 'a', '_i17', '_i18', '_i19', '_i20', 'x', '_i21', '_i22', '_i23', '_i24', 'Fred', '_i25', 'func', '_i26', '_i27'])
dict_keys(['bar'])


In [29]:
# What happens with name clashes?
foo = 1
def func():
    foo = 2 # creates a new variable foo in local scope, global foo in not affected
    print(foo) # prints 2
    # global variable foo still exists, unchanged:
    print(globals()['foo']) # prints 1
    print(locals()['foo']) #prints 2
func()

2
1
2


In [31]:
# To modify a global variable, use keyword global:
foo = 1
def func():
    global foo
    foo = 2 # this modifies the global foo, rather than creating a local variable

In [32]:
# The scope is defined for the whole body of the function
foo = 1
def func():
    # This function has a local variable foo, because it is defined down below.
    # So, foo is local from this point, Global foo is hidden.
    print(foo) # raises UnboundLocalError, because local foo is not yet initialized
    foo = 7
    print(foo)

In [37]:
foo = 1
def func():
    #In this function, foo is a global variable from the beginning
    foo = 7 # global foo is modified
    print(foo) # 7
    print(globals()['foo']) # 7
    # this could be anywhere within the function
    print(foo) # 7
func()

7
1
7


In [38]:
# Functions within functions
foo = 1
def f1():
    bar = 1
    def f2():
        bax = 2
        #here, foo is a global variable, baz is a local variable
        # bar is not in either scope
        print(locals().keys()) #['baz']
        print('bar' in locals()) # False
        print('bar' in globals()) # False
    def f3():
        baz = 3
        print(bar) # bar from f1 is referenced so it enters local scope of f3 (closure)
        print(locals().keys()) # ['bar', 'baz']
        print('bar' in locals()) # True
        print('bar' in globals()) # False
    def f4():
        bar = 4 # a new local bar which hides bar from local scope of f1
        baz = 4
        print(bar)
        print(locals().keys()) # ['bar', 'baz']
        print('bar' in locals()) # True
        print('bar' in globals()) # False

In [39]:
# global vs nonlocal
foo = 0 # global foo
def f1():
    foo = 1 # a new foo local in f1
    def f2():
        foo = 2 # a new foo in local in f2
        def f4():
            global foo
            print(foo) # 0
            foo = 100 # modifies global foo

In [40]:
# for Python 3.x versions
def f1():
    def f2():
        foo = 2
        def f3():
            nonlocal foo # foo from f2, which is the nearest enclosing scope
            print(foo) # 2
            foo = 20 # modifies foo from f2!

In [41]:
# Binding Occurrence
x = 5
x += 7
for x in iterable: pass

NameError: name 'iterable' is not defined